In [1]:
import pandas as pd
from datetime import timedelta,time,datetime
import numpy as np
import matplotlib.pyplot as plt
import copy
from sklearn.utils import shuffle
import datetime as DateTime
import pickle
from utilities import *

In [4]:
chargeSessionFile="data sets/transactions_2018_orig.csv"
intraDayFile='data sets/intraday_amp.xlsx'
dayAheadFile='data sets/Day-ahead Prices_201901010000-202001010000.csv'
save=True
intervalLength=5

In [5]:
#read data files
df=pd.read_csv(chargeSessionFile,delimiter=',')
priceDf=pd.read_csv(dayAheadFile,delimiter=',"')
intraDay=pd.read_excel(intraDayFile)

C:\Users\larsr\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


# Preprocess EV charge sessions data

In [6]:
#convert data types and clean fields
df=df.sort_values(by='Started')
df['Started']=pd.to_datetime(df['Started'])
df['Ended']=pd.to_datetime(df['Ended'])

df['end_time']=df['Ended'].apply(lambda x: x.time())
df['start_time']=df['Started'].apply(lambda x: x.time())
df=df[(df['ChargeTime']>0)&(df['ChargeTime']<=24)]
df['AveragePower']=df['TotalEnergy']/df['ChargeTime']
df['AveragePower']=df['AveragePower'].astype(int)

In [7]:
#investigate different charge patterns
#charged on full capacity
fullCharged1=copy.deepcopy(df[(df['MaxPower']<=df['AveragePower'])&(df['ChargeTime']<df['ConnectedTime'])])

#charged on semi full capacity
fullCharged2=copy.deepcopy(df[(df['MaxPower']>df['AveragePower'])&(df['ChargeTime']<df['ConnectedTime'])])

lackCapacity=copy.deepcopy(df[(df['MaxPower']>df['AveragePower'])&(df['ChargeTime']>=df['ConnectedTime'])])
shortTime=copy.deepcopy(df[(df['MaxPower']<=df['AveragePower'])&(df['ChargeTime']>=df['ConnectedTime'])])

print("charge profiles: ",len(fullCharged1)/len(df),len(fullCharged2)/len(df),len(lackCapacity)/len(df),len(shortTime)/len(df))


charge profiles:  0.02026492684855674 0.6658758402530645 0.2981415579280348 0.01571767497034401


In [8]:
#adjust according to charge profiles
lackCapacity['ChargeTime']=lackCapacity['TotalEnergy']/(lackCapacity['MaxPower'])
fullCharged2['ChargeTime']=fullCharged2['TotalEnergy']/(fullCharged2['MaxPower'])

df=pd.concat([fullCharged1,fullCharged2,lackCapacity,shortTime],axis=0)


In [9]:
#extract jobs that last multiple days
conTime=df[(df['ConnectedTime']>=24)|(df['start_time']>df['end_time'])]
longStays=copy.deepcopy(conTime)
df=df[(df['ConnectedTime']<=24)&(df['start_time']<df['end_time'])]

In [10]:
#cut jobs that last multiple days
newRows=[]
for row in longStays.iterrows():
    if row[1]['Ended'].hour==0 and row[1]['Ended'].minute==0:
        row[1]['Ended']-=timedelta(minutes=1)
        row[1]['TotalEnergy']-=(1/60)*row[1]['MaxPower']
        row[1]['ChargeTime']-=(1/60)
        newRows.append(row[1])
        continue
    firstPart=copy.deepcopy(row[1])
    secondPart=copy.deepcopy(row[1])
        
    firstPart['Ended']=firstPart['Started'].replace(hour=23,minute=59,second=58)
    secondPart['Started']=(firstPart['Ended']+ timedelta(seconds=3))
    
    firstPart['ConnectedTime']=np.round((firstPart['Ended']-firstPart['Started']).seconds/3600,1)
    #minimum connected time of one day
    if secondPart['ConnectedTime']-firstPart['ConnectedTime']>=24:
        secondPart['Ended']=secondPart['Started'].replace(hour=23,minute=59,second=58)
    else:
        secondPart['Ended']=secondPart['Started']+timedelta(hours=secondPart['ConnectedTime']-firstPart['ConnectedTime'])
    
    secondPart['ConnectedTime']=np.round((secondPart['Ended']-secondPart['Started']).seconds/3600,1)
    
    totalConnectedTime=secondPart['ConnectedTime']+firstPart['ConnectedTime']
    totalChargeTime=firstPart['ChargeTime']
    firstPart['ChargeTime']=np.round((firstPart['ConnectedTime']/totalConnectedTime)*totalChargeTime,1)
    secondPart['ChargeTime']=np.round((secondPart['ConnectedTime']/totalConnectedTime)*totalChargeTime,1)
    
    
    firstPart['TotalEnergy']=int(firstPart['MaxPower']*firstPart['ChargeTime'])
    secondPart['TotalEnergy']=int(secondPart['MaxPower']*(secondPart['ChargeTime']))
    newRows.append(firstPart)
    newRows.append(secondPart)
df=pd.concat([df,pd.DataFrame(newRows)],axis=0).reset_index()
df=df[df['ChargeTime']>0]
df=shuffle(df)

In [11]:
#check for valid start and end time
df['end_time']=df['Ended'].apply(lambda x: x.time().replace(microsecond=0))
df['start_time']=df['Started'].apply(lambda x: x.time().replace(microsecond=0))

if len(df[df['start_time']>df['end_time']])>0 or len(df[df['Started']>df['Ended']])>0:
    print("still invalid start and end time",len(df[df['start_time']>df['end_time']]))
    
df=df[df['start_time']<=df['end_time']]
df=df[df['ChargeTime']>0]
df['weekday']=df['Started'].apply(lambda x: x.dayofweek)



In [12]:
#aggregate on 6 weeks
weeks=6
nrs=int(len(df)/weeks)
bound=0
days=[]
dayCount=0
for i in range(weeks):
    subDf=df[bound:bound+nrs]['weekday'].as_matrix()
    bound+=nrs
    for i in subDf:
        days.append(i+dayCount)
    
    dayCount+=7
df=df[0:nrs*6]
df['days_for_aggregation']=days


C:\Users\larsr\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [13]:
#calculate fields
df['start_month']=df['Started'].apply(lambda x: x.month)
df['start_hour']=df['Started'].apply(lambda x: x.hour)
df['start_year']=df['Started'].apply(lambda x: x.year)
df['day']=df['Started'].apply(lambda x: x.day)
df['end minute']=df['end_time'].apply(lambda x: x.minute)

df['connected_time']=df['Ended']-df['Started']
df['charge_timeObj']=df['ChargeTime']*60
df['charge_timeObj']=df['charge_timeObj'].apply(lambda x: timedelta(minutes=int(x)))

df['EndCharge']=df['Started']+df['charge_timeObj']
df['arrival_time']=df['Started']
df['arrival_time']=df['arrival_time'].apply(lambda x:x.replace(second=0))
df['due_time']=df['Ended']

df=df[df['connected_time']>timedelta(minutes=10)]
df['latest_start_time']=df['Ended']-df['charge_timeObj']
df['AveragePowerConnTime']=df['TotalEnergy']/df['ConnectedTime']
try:
    df=df.drop(['index','Unnamed: 0', 'end_time','end minute','connected_time','charge_timeObj'],axis=1)
except:
    print("nothing to drop")
df['MaxPower(KW)']=df['MaxPower']/1000
df['TotalEnergy(KWh)']=df['TotalEnergy']/1000
df['arrival_time']=df['arrival_time'].apply(lambda x: x.time())
df['due_time']=df['due_time'].apply(lambda x: x.time())

In [14]:
#make sure jobs are feasible after rounding
intervals=get_time_intervals(intervalLength)
df['TotalEnergy(KWh)']=df['TotalEnergy(KWh)'].apply(lambda x: np.round(x,2))
df['MaxPower(KW)']=df['MaxPower(KW)'].apply(lambda x: np.round(x,2))

df['arrivaltime_no']=df['arrival_time'].apply(lambda x: convertTimeToInterval(x,intervals))
df['duetime_no']=df['due_time'].apply(lambda x: convertTimeToInterval(x,intervals))
df['deviation']=((df['duetime_no']-df['arrivaltime_no'])*df['MaxPower(KW)']*np.round(5/60,2))-df['TotalEnergy(KWh)']
df['time_till_due']=df['duetime_no']-df['arrivaltime_no']

def applyNegdeviation(x):
    if x['deviation']<0:
        return x['TotalEnergy(KWh)'] + x['deviation']
    else:
        return x['TotalEnergy(KWh)']
df['TotalEnergy(KWh)']=df.apply(lambda x:applyNegdeviation(x),axis=1)

In [15]:
if save:
    df.to_csv("data sets/preproc_charge_sessions.csv")

# Preprocess price data

Day ahead data

In [16]:
#convert fields
priceDf['start_interval']=priceDf['"MTU (CET)'].apply(lambda x:x.split('-')[0][1:])
priceDf['start_interval']=priceDf['"MTU (CET)'].apply(lambda x:x.split('-')[1])
priceDf['price(EUR/MWh)']=priceDf['"Day-ahead Price [EUR/MWh]"""'].apply(lambda x:(x.strip('"')))
priceDf=priceDf.drop(['"MTU (CET)','"Day-ahead Price [EUR/MWh]"""'],axis=1)

In [17]:
#interpolate
newPrices=[]
prices=priceDf['price(EUR/MWh)'].values
for i,price in zip(range(len(prices)),prices):
    if price=='':
        newPrices.append((float(prices[i-1])+float(prices[i+1]))/2)
    else:
        try:
            newPrices.append(float(price))
        except:
            newPrices.append(0)
priceDf['price(EUR/MWh)']=newPrices

In [18]:
#calculate fields
priceDf['price(EUR/KWh)']=priceDf['price(EUR/MWh)']/1000
priceDf['start_interval']=pd.to_datetime(priceDf['start_interval'],yearfirst=True,dayfirst=True)
priceDf['date']=priceDf['start_interval'].apply(lambda x:x.date())
priceDf=priceDf.sort_values('start_interval')
priceDf['day']=priceDf['date'].apply(lambda x: x.weekday())
priceDf['hour']=priceDf['start_interval'].apply(lambda x: x.hour)
priceDf['weekday']=priceDf['date'].apply(lambda x: x.weekday())

#select 42 days
startDate=list(priceDf[(priceDf['day']==0)&(priceDf['hour']==0)].index)
priceDf=priceDf[startDate[0]:(startDate[0]+(42*24))]

In [19]:
#calculate aggregation date
dateToAggDay={}
i=-1
dates=list(priceDf['date'].unique())
dates.sort()
for date in list(priceDf['date'].unique()):
    dateToAggDay[date]=i
    i+=1
priceDf['days_for_aggregation']=priceDf['date'].apply(lambda x:dateToAggDay[x])

Intra day data

In [20]:
#calculate fields
intraDay['date']=intraDay['Product'].apply(lambda x: x[3:-6])
intraDay['date']=pd.to_datetime(intraDay['date'])
intraDay['Product']=intraDay['Product'].apply(lambda x:x if str(x).find('I')==-1 else "None")
intraDay=intraDay[intraDay['Product']!="None"]
intraDay['day']=intraDay['date'].apply(lambda x: x.day)
intraDay['month']=intraDay['date'].apply(lambda x: x.month)
intraDay['year']=intraDay['date'].apply(lambda x: x.year)
intraDay['hour']=intraDay['Product'].apply(lambda x: int(x[12:-3]))
intraDay['hour']=intraDay['hour'].apply(lambda x: 0 if x==24 else x)
intraDay['Avg']=intraDay['Avg'].apply(lambda x: x.replace(',','.'))
intraDay['Avg']=intraDay['Avg'].astype(float)
intraDay=intraDay[intraDay['Avg']!=0]

intraDay['time']=intraDay['hour'].apply(lambda x: DateTime.time(x,0,0))
intraDay['datetime']=pd.to_datetime(intraDay['date'].astype(str)+ ' ' + intraDay['time'].astype(str))
intraDay=intraDay.sort_values(by='datetime')



In [21]:
#fill missing value with value of previous hour
newRows=[]
for month in list(intraDay['month'].unique()):
    for day in list(intraDay[intraDay['month']==month]['day'].unique()):
        for hour in range(24):
            if hour not in list(intraDay[(intraDay['month']==month)&(intraDay['day']==day)]['hour'].values):
                newRow=intraDay[(intraDay['month']==month)&(intraDay['day']==day)&(intraDay['hour'].isin([hour-1]))][0:1]
                newRow['hour']=hour
                intraDay=intraDay.append(newRow,ignore_index=True)


In [22]:
#calculate fields
intraDay['time']=intraDay['hour'].apply(lambda x: DateTime.time(x,0,0))
intraDay['datetime']=pd.to_datetime(intraDay['date'].astype(str)+ ' ' + intraDay['time'].astype(str))
intraDf=intraDay.sort_values(by='datetime')
try:
    priceDf=priceDf.reset_index()
except:
    print("already reset")
try:
    intraDf=intraDf.reset_index()
except:
    print("already reset")
intraDf['weekday']=intraDf['datetime'].apply(lambda x: x.weekday())
intraDf['days_for_aggregation']=priceDf['days_for_aggregation']+1
intraDf['price(EUR/KWh)']=intraDf['Avg']/1000

In [23]:
if save:
    priceDf.to_csv("data sets/day_ahead_price_pre_proc.csv")
    intraDf.to_csv("data sets/intra_day_price_pre_proc.csv")